# AHU Sizes merging

Import all the modules

In [1]:
import pandas as pd
from pandas import ExcelWriter

Import Excel files

In [2]:
# Units with the sizes inside
excel_file = 'Units_modified_20200204.xlsx'
df_units = pd.read_excel(excel_file)
df_units.sort_values(by=['File name', 'Line'])
df_units.head()

,File name,Line,AHU
0,0004031240-,10,DV10
1,0004390352-rev 2,10,DV50
2,0004390352-rev 2,20,DV25
3,0004390352-rev 2,30,DV40
4,0004535899- Residencia Gent Gran Gracia DVF,10,DV20


In [3]:
# Units with the sizes inside
excel_file = 'Total Results_modified.xlsx'
df_results = pd.read_excel(excel_file)
df_results['Line'] = 0
df_results.sort_values(by=['File name', 'Page'])
df_results.head()

,Page,Airflow,Static Press.,No Fans,Motor Power,RPM,ID,Gross,Consump. kW,File name,Line
0,11,12780.0,656,1,5.0,1750,5.00-1750,845.098022,3.67,0004390352-rev 2,0
1,12,12780.0,647,1,5.0,1750,5.00-1750,845.098022,3.63,0004390352-rev 2,0
2,34,5760.0,588,1,2.4,2400,2.40-2400,505.252014,1.53,0004390352-rev 2,0
3,36,5760.0,577,1,2.4,2400,2.40-2400,505.252014,1.51,0004390352-rev 2,0
4,58,10080.0,669,1,5.4,2130,5.40-2130,787.940979,3.11,0004390352-rev 2,0


Assign the number of line accordingly

In [4]:
list_unique = df_results['File name'].unique()
max_sep = 5

for order in list_unique:
    total_no_fans = len(df_results.loc[df_results['File name'] == order, 'Page'])
    df_results.loc[df_results['File name'] == order]

    if total_no_fans == 1:
        df_results.loc[df_results['File name'] == order, 'Line'] = 10
    elif total_no_fans == 2:
        first_page = df_results.loc[df_results['File name'] == order, 'Page'].iloc[0]
        second_page = df_results.loc[df_results['File name'] == order, 'Page'].iloc[1]
        diff = second_page - first_page
        if diff < max_sep:
            df_results.loc[df_results['File name'] == order, 'Line'] = 10
        else:
            lines = [10, 20]
            df_results.loc[df_results['File name'] == order, 'Line'] = lines
    elif total_no_fans > 2:
        # First fan is first fan always
        former_page = df_results.loc[df_results['File name'] == order, 'Page'].iloc[0]
        lines = [10]

        # Check the rest of fans
        for page in range(1, total_no_fans):
            next_page = df_results.loc[df_results['File name'] == order, 'Page'].iloc[page]
            diff = next_page - former_page
            former_line = lines[-1:][0]
            if diff < max_sep:
                lines.append(former_line)
            else:
                lines.append(former_line+10)
            # Reset the page so that it does not affect the next page in the loop
            former_page = next_page

        # Merge the list with the dataframe
        df_results.loc[df_results['File name'] == order, 'Line'] = lines
    
print('Done!')

'''
-----------------------------------------------------------------------------------------
NOTE: ".loc" + ".iloc" works fine for extracting the value from the dataframe, but it really sucks at assigning it.
value = df_results.loc[df_results['File name'] == order, 'Page'].iloc[0] # This works flawlessly
df_results.loc[df_results['File name'] == order, 'Page'].iloc[0] = 10 # This does not

One should use a vector instead and pass it to the entire filtered dataframe
values = [10, 20, 30, 50]
df_results.loc[df_results['File name'] == order, 'Page'] = values
-----------------------------------------------------------------------------------------
'''

Done!


'\n-----------------------------------------------------------------------------------------\nNOTE: ".loc" + ".iloc" works fine for extracting the value from the dataframe, but it really sucks at assigning it.\nvalue = df_results.loc[df_results[\'File name\'] == order, \'Page\'].iloc[0] # This works flawlessly\ndf_results.loc[df_results[\'File name\'] == order, \'Page\'].iloc[0] = 10 # This does not\n\nOne should use a vector instead and pass it to the entire filtered dataframe\nvalues = [10, 20, 30, 50]\ndf_results.loc[df_results[\'File name\'] == order, \'Page\'] = values\n-----------------------------------------------------------------------------------------\n'

Merge both dataframes

In [5]:
new_df = pd.merge(df_results, df_units, how='right', on=['File name', 'Line'])
new_df.head()

,Page,Airflow,Static Press.,No Fans,Motor Power,RPM,ID,Gross,Consump. kW,File name,Line,AHU
0,11.0,12780.0,656.0,1.0,5.0,1750.0,5.00-1750,845.098022,3.67,0004390352-rev 2,10,DV50
1,12.0,12780.0,647.0,1.0,5.0,1750.0,5.00-1750,845.098022,3.63,0004390352-rev 2,10,DV50
2,34.0,5760.0,588.0,1.0,2.4,2400.0,2.40-2400,505.252014,1.53,0004390352-rev 2,20,DV25
3,36.0,5760.0,577.0,1.0,2.4,2400.0,2.40-2400,505.252014,1.51,0004390352-rev 2,20,DV25
4,58.0,10080.0,669.0,1.0,5.4,2130.0,5.40-2130,787.940979,3.11,0004390352-rev 2,30,DV40


Export to Excel

In [6]:
# Export to Excel
name = 'Merged Results.xlsx'
writer = pd.ExcelWriter(name)
new_df.to_excel(writer, index = False)
writer.save()
